In [1]:
from ioh import get_problem
import numpy as np
from ioh import problem, OptimizationType
from ioh import Experiment
import ioh
from ioh import logger
from nevergrad import functions
import nevergrad as ng

In [2]:
def wrap_ng_func(f, dim):    #wrapping single nevergrad problem/objective function
    def f_temp(x):
        return f(np.array(x))
    f_ng = problem.wrap_real_problem(f_temp, f.__name__, n_variables=dim, optimization_type=OptimizationType.Minimization)
    return f_ng

In [3]:
names = [
        "hm",
        "rastrigin",
        "griewank",
        "rosenbrock",
        "ackley",
        "lunacek",
        "deceptivemultimodal",
        "bucherastrigin",
        "multipeak",
    ]
names += ["sphere", "doublelinearslope"]  #, "stepdoublelinearslope"]
#names += ["cigar", "altcigar", "ellipsoid", "altellipsoid", "stepellipsoid", "discus", "bentcigar"]
#names += ["deceptiveillcond", "deceptivemultimodal", "deceptivepath"]   #list of objective functions

In [4]:
def wrap_problems_yabbob(dim):   #wrapping all YABBOB suite problems
    problems = [
        wrap_ng_func(functions.corefuncs.hm, dim),
        wrap_ng_func(functions.corefuncs.rastrigin, dim),
        wrap_ng_func(functions.corefuncs.griewank, dim),
        wrap_ng_func(functions.corefuncs.rosenbrock, dim),
        wrap_ng_func(functions.corefuncs.ackley, dim),
        wrap_ng_func(functions.corefuncs.lunacek, dim),
        wrap_ng_func(functions.corefuncs.deceptivemultimodal, dim),
        wrap_ng_func(functions.corefuncs.bucherastrigin, dim),
        wrap_ng_func(functions.corefuncs.multipeak, dim),
        wrap_ng_func(functions.corefuncs.sphere, dim), wrap_ng_func(functions.corefuncs.doublelinearslope, dim),
        #wrap_ng_func(functions.corefuncs.stepdoublelinearslope, dim)
        #wrap_ng_func(functions.corefuncs.cigar, dim), wrap_ng_func(functions.corefuncs.altcigar, dim),
        #wrap_ng_func(functions.corefuncs.ellipsoid, dim), wrap_ng_func(functions.corefuncs.altellipsoid, dim),
        #wrap_ng_func(functions.corefuncs.stepellipsoid, dim),
        #wrap_ng_func(functions.corefuncs.discus, dim), wrap_ng_func(functions.corefuncs.bentcigar, dim),
        #wrap_ng_func(functions.corefuncs.deceptiveillcond, dim), wrap_ng_func(functions.corefuncs.deceptivemultimodal, dim),
        #wrap_ng_func(functions.corefuncs.deceptivepath, dim)
    ]
    return problems

In [5]:
#problems = wrap_problems_yabbob(5)

In [6]:
#algorithms = ["CMandAS2", "ECMA", "Cobyla", "MetaModel", "CMA", "DiagonalCMA", "FCMA", "BOBYQA", "F127CMA", "Powell", "SQP"]
algorithms = ["CMA", "DiagonalCMA", "FCMA"]

In [7]:
budgets = [10, 100, 1000]
dimension = 20 #[5,10,20,100]

In [8]:
def param(dimension):
    x = ng.p.Array(shape=(dimension,))
    parametrization = ng.p.Instrumentation(x)
    return parametrization

In [ ]:
problems = wrap_problems_yabbob(dimension)
for f in problems:
    name_idx = problems.index(f)
    name = names[name_idx]
    for runs in range(10):
        l = logger.Analyzer(root="data_"+name, folder_name="run_"+str(runs))
        f.attach_logger(l)
        for budget in budgets:
            for algo in algorithms:
            
                optim = ng.optimizers.registry[algo](parametrization=param(dimension), budget=budget)
                for u in range(budget):
                    x1 = optim.ask()
                        # Ask and tell can be asynchronous.
                        # Just be careful that you "tell" something that was asked.
                        # Here we ask 3 times and tell 3 times in order to fake asynchronicity
                    x2 = optim.ask()
                    x3 = optim.ask()
                        # The three folowing lines could be parallelized.
                        # We could also do things asynchronously, i.e. do one more ask
                        # as soon as a training is over.
                    y1 = f(*x1.args, **x1.kwargs)  # here we only defined an arg, so we could omit kwargs
                    y2 = f(*x2.args, **x2.kwargs)  # (keeping it here for the sake of consistency)
                    y3 = f(*x3.args, **x3.kwargs)
                    optim.tell(x1, y1)
                    optim.tell(x2, y2)
                    optim.tell(x3, y3)
                recommendation = optim.recommend()
                print("* ", "Dimension:", dimension, "Problem:",  name, "Run:", runs, "Budget:", budget, "Algorithm:", algo, 
                      " provides a vector of parameters with test error ",
                      f(*recommendation.args, **recommendation.kwargs))

(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:40:14 2021)
*  Dimension: 20 Problem: hm Run: 0 Budget: 10 Algorithm: CMA  provides a vector of parameters with test error  11.766484611650295
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:40:14 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: hm Run: 0 Budget: 10 Algorithm: DiagonalCMA  provides a vector of parameters with test error  12.32500383955012
*  Dimension: 20 Problem: hm Run: 0 Budget: 10 Algorithm: FCMA  provides a vector of parameters with test error  19.375429265131693
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:40:14 2021)
*  Dimension: 20 Problem: hm Run: 0 Budget: 100 Algorithm: CMA  provides a vector of parameters with test error  0.48291414089189966
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:40:14 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: hm Run: 0 Budget: 

*  Dimension: 20 Problem: hm Run: 4 Budget: 100 Algorithm: FCMA  provides a vector of parameters with test error  0.3650574971413998
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:40:39 2021)
*  Dimension: 20 Problem: hm Run: 4 Budget: 1000 Algorithm: CMA  provides a vector of parameters with test error  0.00993110235974249
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:40:41 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: hm Run: 4 Budget: 1000 Algorithm: DiagonalCMA  provides a vector of parameters with test error  0.039723249156955125
*  Dimension: 20 Problem: hm Run: 4 Budget: 1000 Algorithm: FCMA  provides a vector of parameters with test error  0.02979300620184764
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:40:44 2021)
*  Dimension: 20 Problem: hm Run: 5 Budget: 10 Algorithm: CMA  provides a vector of parameters with test error  16.89692019920109
(6_w,12)-aCMA-ES (mu_w=3.

*  Dimension: 20 Problem: hm Run: 9 Budget: 100 Algorithm: CMA  provides a vector of parameters with test error  0.6919551197729092
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:41:10 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: hm Run: 9 Budget: 100 Algorithm: DiagonalCMA  provides a vector of parameters with test error  0.4463970227848245
*  Dimension: 20 Problem: hm Run: 9 Budget: 100 Algorithm: FCMA  provides a vector of parameters with test error  0.3855359480268649
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:41:10 2021)
*  Dimension: 20 Problem: hm Run: 9 Budget: 1000 Algorithm: CMA  provides a vector of parameters with test error  0.019861819012029564
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:41:12 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: hm Run: 9 Budget: 1000 Algorithm: DiagonalCMA  provides a vector of parameters with test error  0.029

*  Dimension: 20 Problem: rastrigin Run: 3 Budget: 1000 Algorithm: CMA  provides a vector of parameters with test error  34.825473617596295
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:41:33 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: rastrigin Run: 3 Budget: 1000 Algorithm: DiagonalCMA  provides a vector of parameters with test error  33.838697941700545
*  Dimension: 20 Problem: rastrigin Run: 3 Budget: 1000 Algorithm: FCMA  provides a vector of parameters with test error  26.863871771685588
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:41:36 2021)
*  Dimension: 20 Problem: rastrigin Run: 4 Budget: 10 Algorithm: CMA  provides a vector of parameters with test error  138.00014182968346
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:41:36 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: rastrigin Run: 4 Budget: 10 Algorithm: DiagonalCMA  provides a vector of p

*  Dimension: 20 Problem: rastrigin Run: 8 Budget: 100 Algorithm: CMA  provides a vector of parameters with test error  141.9945491697332
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:41:57 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: rastrigin Run: 8 Budget: 100 Algorithm: DiagonalCMA  provides a vector of parameters with test error  141.01894731393637
*  Dimension: 20 Problem: rastrigin Run: 8 Budget: 100 Algorithm: FCMA  provides a vector of parameters with test error  135.24997735814011
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:41:58 2021)
*  Dimension: 20 Problem: rastrigin Run: 8 Budget: 1000 Algorithm: CMA  provides a vector of parameters with test error  24.875831280270127
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:41:59 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: rastrigin Run: 8 Budget: 1000 Algorithm: DiagonalCMA  provides a vector of p

...

*  Dimension: 20 Problem: rosenbrock Run: 1 Budget: 1000 Algorithm: CMA  provides a vector of parameters with test error  14.788891046273278
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:43:15 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: rosenbrock Run: 1 Budget: 1000 Algorithm: DiagonalCMA  provides a vector of parameters with test error  19.453676989234154
*  Dimension: 20 Problem: rosenbrock Run: 1 Budget: 1000 Algorithm: FCMA  provides a vector of parameters with test error  17.01865068958751
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:43:18 2021)
*  Dimension: 20 Problem: rosenbrock Run: 2 Budget: 10 Algorithm: CMA  provides a vector of parameters with test error  1240.7747733243928
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:43:18 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: rosenbrock Run: 2 Budget: 10 Algorithm: DiagonalCMA  provides a vector 

*  Dimension: 20 Problem: rosenbrock Run: 6 Budget: 100 Algorithm: CMA  provides a vector of parameters with test error  92.0941371511277
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:43:41 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: rosenbrock Run: 6 Budget: 100 Algorithm: DiagonalCMA  provides a vector of parameters with test error  121.7376257711632
*  Dimension: 20 Problem: rosenbrock Run: 6 Budget: 100 Algorithm: FCMA  provides a vector of parameters with test error  158.81282665736398
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:43:42 2021)
*  Dimension: 20 Problem: rosenbrock Run: 6 Budget: 1000 Algorithm: CMA  provides a vector of parameters with test error  13.919413136794823
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:43:44 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: rosenbrock Run: 6 Budget: 1000 Algorithm: DiagonalCMA  provides a vector o

*  Dimension: 20 Problem: ackley Run: 0 Budget: 1000 Algorithm: CMA  provides a vector of parameters with test error  1.0058703364013155e-05
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:44:07 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: ackley Run: 0 Budget: 1000 Algorithm: DiagonalCMA  provides a vector of parameters with test error  1.1866283036177094e-06
*  Dimension: 20 Problem: ackley Run: 0 Budget: 1000 Algorithm: FCMA  provides a vector of parameters with test error  8.438051257275703e-06
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:44:10 2021)
*  Dimension: 20 Problem: ackley Run: 1 Budget: 10 Algorithm: CMA  provides a vector of parameters with test error  4.379970912105254
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:44:10 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: ackley Run: 1 Budget: 10 Algorithm: DiagonalCMA  provides a vector of parame

*  Dimension: 20 Problem: ackley Run: 5 Budget: 100 Algorithm: CMA  provides a vector of parameters with test error  2.033797382977593
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:44:33 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: ackley Run: 5 Budget: 100 Algorithm: DiagonalCMA  provides a vector of parameters with test error  2.0464472107580716
*  Dimension: 20 Problem: ackley Run: 5 Budget: 100 Algorithm: FCMA  provides a vector of parameters with test error  2.3397695411664405
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:44:33 2021)
*  Dimension: 20 Problem: ackley Run: 5 Budget: 1000 Algorithm: CMA  provides a vector of parameters with test error  4.873071237643245e-06
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:44:35 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: ackley Run: 5 Budget: 1000 Algorithm: DiagonalCMA  provides a vector of parameters wi

*  Dimension: 20 Problem: ackley Run: 9 Budget: 1000 Algorithm: DiagonalCMA  provides a vector of parameters with test error  2.283098375688297e-06
*  Dimension: 20 Problem: ackley Run: 9 Budget: 1000 Algorithm: FCMA  provides a vector of parameters with test error  8.505310728157411e-06
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:45:00 2021)
*  Dimension: 20 Problem: lunacek Run: 0 Budget: 10 Algorithm: CMA  provides a vector of parameters with test error  255.13859266087817
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:45:00 2021)
   Covariance matrix is diagonal
*  Dimension: 20 Problem: lunacek Run: 0 Budget: 10 Algorithm: DiagonalCMA  provides a vector of parameters with test error  248.77117131435898
*  Dimension: 20 Problem: lunacek Run: 0 Budget: 10 Algorithm: FCMA  provides a vector of parameters with test error  278.7362071461281
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 20 (seed=nan, Fri Oct  1 00:45:00 2